In [1]:
import pandas as pd
import pymysql
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
_db = pymysql.connect(
    host=os.getenv('host'),
    user=os.getenv('user'),
    password=os.getenv('pwd'),
    db=os.getenv('db_name'),
    port=int(os.getenv('port'))
)

In [4]:
cursor = _db.cursor(pymysql.cursors.DictCursor)

In [ ]:
select_sql2 = "select * from `sales records`"

cursor.execute(select_sql2)

result = cursor.fetchall()

df = pd.DataFrame(result)


# order date 시계열로 변경
df['Order Date'] = pd.to_datetime(df['Order Date'], format="%m/%d/%Y")

# 새로운 컬럼 order year 에 년도 저장
df['Order Year'] = [str(date)[:4] for date in df['Order Date']]

# sales channel과 order year를 기준으로 그룹화하여 total profit 합
group_data = df.groupby(["Sales Channel", "Order Year"])['Total Profit'].sum()

# offline / online data 추출
offline_flag = [x[0] == 'Offline' for x in group_data.index]
online_flag = [x[0] == 'Online' for x in group_data.index]
offline_data = group_data.loc[offline_flag]
online_data = group_data.loc[online_flag]

# online index 추출
online_index = list(online_data.index)
# offline index 추출
offline_index = list(offline_data.index)
# online value 추출
online_value = online_data.to_list()
# offline value 추출
offline_value = offline_data.to_list()
# 두 그래프 생성
print(online_index, online_value)

[('Online', '2010'), ('Online', '2011'), ('Online', '2012'), ('Online', '2013'), ('Online', '2014'), ('Online', '2015'), ('Online', '2016'), ('Online', '2017')] [249729397.48, 257630896.77, 243948685.59, 257982751.1, 261883913.38, 303531083.86, 245531172.42, 149753402.54]


In [9]:
select_sql2 = "select * from `sales records` where `item type` in (%s,%s)"
values = ["vegetables","baby food"]

cursor.execute(select_sql2, values)

1678

In [ ]:
def select_query(_sql, *_val):
    cursor.execute(_sql, _val)
    data = cursor.fetchall()
    df = pd.DataFrame(data)
    return df
    
select_sql3 = "select * from `sales records` where `item type` in (%s,%s,%s)"
select_query(select_sql3, "meat", "cereal", "clothes")

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Sub-Saharan Africa,Eritrea,Meat,Online,L,11/21/2014,411809480,1/10/2015,2431,421.89,364.69,1025614.59,886561.39,139053.20
1,Sub-Saharan Africa,Zambia,Cereal,Offline,M,7/26/2016,773452794,8/24/2016,724,205.70,117.11,148926.80,84787.64,64139.16
2,Middle East and North Africa,Algeria,Clothes,Offline,C,6/21/2011,181401288,7/21/2011,9527,109.28,35.84,1041110.56,341447.68,699662.88
3,Europe,Russia,Meat,Offline,L,12/2/2015,305029237,12/26/2015,3784,421.89,364.69,1596431.76,1379986.96,216444.80
4,Europe,Greece,Clothes,Online,C,10/9/2016,510666692,10/13/2016,6477,109.28,35.84,707806.56,232135.68,475670.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2490,North America,Mexico,Cereal,Online,M,7/4/2017,783641326,8/18/2017,5737,205.70,117.11,1180100.90,671860.07,508240.83
2491,Asia,Kazakhstan,Clothes,Online,L,3/15/2012,691468608,4/25/2012,9782,109.28,35.84,1068976.96,350586.88,718390.08
2492,Sub-Saharan Africa,Angola,Meat,Online,H,3/4/2014,640492904,4/14/2014,4938,421.89,364.69,2083292.82,1800839.22,282453.60
2493,Central America and the Caribbean,Saint Lucia,Meat,Offline,M,11/4/2011,982547509,12/15/2011,6497,421.89,364.69,2741019.33,2369390.93,371628.40


In [25]:
_db.commit()

In [28]:
sql_query = "select * from `emp` where `sal` >= 1500"
df = select_query(sql_query)
df

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
2,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
3,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0
4,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
5,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0
6,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
7,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,0.0,20.0


In [29]:
sql_query = "select * from `emp`"
df = select_query(sql_query)
df

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0


In [34]:
flag = df['SAL'] >= 1500
df.loc[flag]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
12,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,0.0,20.0


In [36]:
sql_query = "SELECT `ENAME` FROM `EMP` WHERE `SAL` BETWEEN 1500 AND 3000"
select_query(sql_query)

,ENAME
0,ALLEN
1,JONES
2,BLAKE
3,CLARK
4,SCOTT
5,TURNER
6,FORD


In [75]:
flag2 = (df['SAL'] >= 1500) & (df['SAL'] <= 3000)
df.loc[flag2]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
12,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,0.0,20.0


In [53]:
sql_query4 = 'select * from `emp` where `ENAME` LIKE "S%%"'
select_query(sql_query4)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0


In [93]:
flag3 = df['ENAME'].str.startswith("S")
flag4 = df['ENAME'].str[0] == "S"
df.loc[flag4]

flag5 = list(map(lambda x : x.startswith("S"), df['ENAME']))
df.loc[flag5]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0


In [98]:
sql_query5 = """
    SELECT A.`ENAME`, A.`JOB`, A.`SAL`
    FROM `EMP` AS A
    LEFT JOIN `DEPT` AS B
    ON a.`DEPTNO` = b.`DEPTNO`
"""

select_query(sql_query5)

,ENAME,JOB,SAL
0,SMITH,CLERK,800.0
1,ALLEN,SALESMAN,1600.0
2,WARD,SALESMAN,1250.0
3,JONES,MANAGER,2975.0
4,MARTIN,SALESMAN,1250.0
5,BLAKE,MANAGER,2850.0
6,CLARK,MANAGER,2450.0
7,SCOTT,ANALYST,3000.0
8,KING,PRESIDENT,5000.0
9,TURNER,SALESMAN,1500.0


In [100]:
emp_select = "select * from emp"
dept_select = "select * from dept"
df1 = select_query(emp_select)
df2 = select_query(dept_select)

In [102]:
print(df1)
print(df2)

     EMPNO   ENAME        JOB     MGR    HIREDATE     SAL    COMM  DEPTNO
0   7369.0   SMITH      CLERK  7902.0  1980-12-17   800.0     0.0    20.0
1   7499.0   ALLEN   SALESMAN  7698.0  1981-02-20  1600.0   300.0    30.0
2   7521.0    WARD   SALESMAN  7698.0  1981-02-22  1250.0   500.0    30.0
3   7566.0   JONES    MANAGER  7839.0  1981-04-02  2975.0     0.0    20.0
4   7654.0  MARTIN   SALESMAN  7698.0  1981-09-28  1250.0  1400.0    30.0
5   7698.0   BLAKE    MANAGER  7839.0  1981-05-01  2850.0     0.0    30.0
6   7782.0   CLARK    MANAGER  7839.0  1981-06-09  2450.0     0.0    10.0
7   7788.0   SCOTT    ANALYST  7566.0  1987-06-28  3000.0     0.0    20.0
8   7839.0    KING  PRESIDENT     0.0  1981-11-17  5000.0     0.0    10.0
9   7844.0  TURNER   SALESMAN  7698.0  1981-09-08  1500.0     0.0    30.0
10  7876.0   ADAMS      CLERK  7788.0  1987-07-13  1100.0     0.0    20.0
11  7900.0   JAMES      CLERK  7698.0  1981-12-03   950.0     0.0    30.0
12  7902.0    FORD    ANALYST  7566.0 

In [107]:
pd.merge(df1, df2, on="DEPTNO", how="left")
df1.merge(df2, on = "DEPTNO", how="left")

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,DNAME,LOC
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,RESEARCH,DALLAS
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,SALES,CHICAGO
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,SALES,CHICAGO
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,RESEARCH,DALLAS
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,SALES,CHICAGO
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,SALES,CHICAGO
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,ACCOUNTING,NEW YORK
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,RESEARCH,DALLAS
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,ACCOUNTING,NEW YORK
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,SALES,CHICAGO


In [108]:
_db2 = pymysql.connect(
    host = "172.30.1.56",
    user="ubion",
    password=os.getenv('pwd'),
    db=os.getenv('db_name'),
    port=int(os.getenv('port'))
)

In [109]:
cursor = _db2.cursor(pymysql.cursors.DictCursor)

In [110]:
query1 = "select * from tran_1"
cursor.execute(query1)

5000

In [111]:
df1 = select_query(query1)
df1

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865
...,...,...,...,...
4995,T0000005108,210000,2019-06-15 02:42:41,HD315748
4996,T0000005109,150000,2019-06-15 03:36:16,HI215420
4997,T0000005110,50000,2019-06-15 03:44:06,IK880102
4998,T0000005111,210000,2019-06-15 04:14:06,IK074758


In [ ]:
import pymysql
import pymysql.cursors

class MyDB:
    def __init__(self, 
                 _host = "127.0.0.1", 
                 _port = 3306, 
                 _user = "root", 
                 _pwd = "1234", 
                 _db_name = "ubion"):
        self.host = _host
        self.port = _port
        self.user = _user
        self.pwd = _pwd
        self.db = _db_name
        
    def connect_sql(self):
        # 서버 연결
        self.server = pymysql.connect(
            host=self.host,
            port=self.port
            user=self.user,
            password=self.pwd,
            db=self.db
        )
        
        # 커서 만들기
        self.cursor = self.server.cursor(pymysql.cursors.DictCursor)
        
    def close_sql(self):
        # 서버 종료
        self.server.close()
        
    def execute_query(self, sql_query, *values):
        self.cursor.execute(sql_query, values)
        df = pd.DataFrame(self.cursor.fetchAll())
        return df
        
        